# RunnableLambda

`RunnableLambda` 는 **사용자 정의 함수를 실행** 할 수 있는 기능을 제공합니다.

이를 통해 개발자는 **자신만의 함수를 정의**하고, 해당 함수를 `RunnableLambda` 를 사용하여 실행할 수 있습니다.

예를 들어, 데이터 전처리, 계산, 또는 외부 API와의 상호 작용과 같은 작업을 수행하는 함수를 정의하고 실행할 수 있습니다.


## 사용자 정의 함수를 실행하는 방법

**주의사항**

사용자 정의 함수를 `RunnableLambda`로 래핑하여 활용할 수 있는데, 여기서 주의할 점은 **사용자 정의 함수가 받을 수 있는 인자는 1개 뿐이라는 점** 입니다.

만약, 여러 인수를 받는 함수로 구현하고 싶다면, 단일 입력을 받아들이고 이를 여러 인수로 풀어내는 래퍼를 작성해야 합니다.


In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [2]:
from operator import itemgetter

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI


def length_function(text):  # 텍스트의 길이를 반환하는 함수
    return len(text)


def _multiple_length_function(text1, text2):  # 두 텍스트의 길이를 곱하는 함수
    return len(text1) * len(text2)


def multiple_length_function(  # 2개 인자를 받는 함수로 연결하는 wrapper 함수
    _dict,
):  # 딕셔너리에서 "text1"과 "text2"의 길이를 곱하는 함수
    return _multiple_length_function(_dict["text1"], _dict["text2"])


# 프롬프트 템플릿 생성
prompt = ChatPromptTemplate.from_template("what is {a} + {b}?")
# ChatOpenAI 모델 초기화
model = ChatOpenAI()

# 프롬프트와 모델을 연결하여 체인 생성
chain1 = prompt | model

# 체인 구성
chain = (
    {
        "a": itemgetter("input_1") | RunnableLambda(length_function),
        "b": {"text1": itemgetter("input_1"), "text2": itemgetter("input_2")}
        | RunnableLambda(multiple_length_function),
    }
    | prompt
    | model
    | StrOutputParser()
)

chain 을 실행하여 결과를 확인합니다.


In [3]:
# 주어진 인자들로 체인을 실행합니다.
chain.invoke({"input_1": "bar", "input_2": "gah"})

'3 + 9 is equal to 12.'

In [6]:
chain.get_graph().print_ascii()

                      +--------------------+                        
                      | Parallel<a,b>Input |                        
                      +--------------------+                        
                       ***                 ****                     
                    ***                        ****                 
                  **                               ****             
+----------------------------+                         **           
| Parallel<text1,text2>Input |                          *           
+----------------------------+                          *           
          **        **                                  *           
        **            **                                *           
       *                *                               *           
+--------+          +--------+                          *           
| Lambda |          | Lambda |                          *           
+--------+          +--------+    

## RunnableConfig 인자로 활용

`RunnableLambda` 는 선택적으로 [RunnableConfig](https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.config.RunnableConfig.html#langchain_core.runnables.config.RunnableConfig) 를 수용할 수 있습니다.

이를 통해 콜백, 태그 및 기타 구성 정보를 중첩된 실행에 전달할 수 있습니다.


In [7]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableConfig
import json


def parse_or_fix(text: str, config: RunnableConfig):
    # 다음 텍스트를 수정하는 프롬프트 템플릿을 생성합니다.
    fixing_chain = (
        ChatPromptTemplate.from_template(
            "Fix the following text:\n\ntext\n{input}\n\nError: {error}"
            " Don't narrate, just respond with the fixed data."
        )
        | ChatOpenAI()
        | StrOutputParser()
    )
    # 최대 3번 시도합니다.
    for _ in range(3):
        try:
            # JSON 형식으로 텍스트를 파싱합니다.
            return json.loads(text)
        except Exception as e:
            # 파싱 중 오류가 발생하면 수정 체인을 호출하여 텍스트를 수정합니다.
            text = fixing_chain.invoke({"input": text, "error": e}, config)
            print(f"config: {config}")
    # 파싱에 실패한 경우 "Failed to parse" 문자열을 반환합니다.
    return "Failed to parse"

In [8]:
from langchain.callbacks import get_openai_callback

with get_openai_callback() as cb:
    # RunnableLambda를 사용하여 parse_or_fix 함수를 호출합니다.
    output = RunnableLambda(parse_or_fix).invoke(
        input="{foo:: bar}",
        config={"tags": ["my-tag"], "callbacks": [cb]},  # config 를 전달합니다.
    )
    # 수정한 결과를 출력합니다.
    print(f"\n\n수정한결과:\n{output}")

config: {'tags': ['my-tag'], 'metadata': {}, 'callbacks': <langchain_core.callbacks.manager.CallbackManager object at 0x7faefc486490>, 'recursion_limit': 25, 'configurable': {}}


수정한결과:
{'foo': 'bar'}


In [9]:
output

{'foo': 'bar'}